# Case 1: No reserve requirements

In [1]:
using CSV, DataFrames, JuMP, HiGHS, SCIP, Clp

storageinfo = CSV.read(joinpath("cleaned","storageinfo.csv"), DataFrame)
generatorinfo = CSV.read(joinpath("cleaned","generatorinfo.csv"), DataFrame)
vreinfo = CSV.read(joinpath("cleaned","vreinfo.csv"), DataFrame)
halfhourlydemand = CSV.read(joinpath("cleaned","halfhourlydemand.csv"), DataFrame)
halfhourlyvrecf = CSV.read(joinpath("cleaned","halfhourlyvrecf.csv"), DataFrame)
halfhourlydemand = halfhourlydemand[1:1440,:];

## Scheme 1 (No storage)

In [2]:
function case1scheme1(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
    budget, voll, bigm, epsilon)

    # SETS
    V = vreinfo.id
    G = generatorinfo.id
    S = storageinfo.id
    T = halfhourlydemand.hh
    T1 = T[2:end]

    # INITIATE MODEL
    model = Model()
    set_optimizer(model, HiGHS.Optimizer)

    # DECISION VARIABLES
    @variables(model, begin
        CAPG[G] >= 0
        GEN[G,T] >= 0
        CURTAIL[V,T] >= 0
        NSE[T] >= 0
    end)

    # BUDGET CONSTRAINT
    @expression(model, GenInvCost, sum(generatorinfo[generatorinfo.id.==g,:fixedCost][1] * CAPG[g] for g in G))
    @expression(model, TIC, GenInvCost)
    @constraint(model, cBudget, TIC <= budget)

    # OBJECTIVE FUNCTION
    @expression(model, EnergyProvisionCost, 
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * GEN[g,t] for g in G for t in T))
    @expression(model, UnservedEnergyCost, 
        sum(0.5 * voll * NSE[t] for t in T))
    @expression(model, CurtailedVreCost,
        sum(0.5 * vreinfo[vreinfo.id.==v,:varCost][1] * CURTAIL[v,t] for v in V for t in T))
    @objective(model, Min, 
        EnergyProvisionCost + UnservedEnergyCost + CurtailedVreCost)

    # POWER BALANCE CONSTRAINT
    @constraint(model, c01[t in T], sum(GEN[g,t] for g in G) + NSE[t] - sum(CURTAIL[v,t] for v in V) +
        sum(vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1] for v in V) == 
        halfhourlydemand[halfhourlydemand.hh.==t,:load][1])
    
    # GENERATOR LIMIT CONSTRAINT
    @constraint(model, c90[g in G, t in T], GEN[g,t] <= CAPG[g])

    # GENERATOR RAMP UP CONSTRAINT
    @constraint(model, c11[g in G, t in T1], 
        GEN[g,t] - GEN[g,t-1] <= generatorinfo[generatorinfo.id.==g,:rampupRate][1] * CAPG[g])
    
    # GENERATOR RAMP DOWN CONSTRAINT
    @constraint(model, c12[g in G, t in T1],
        GEN[g,t-1] - GEN[g,t] <= generatorinfo[generatorinfo.id.==g,:rampdownRate][1] * CAPG[g])
    
    # VRE CURTAIL CONSTRAINT
    @constraint(model, c13[v in V, t in T], CURTAIL[v,t] <= 
        vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1])

    optimize!(model)

    return(
        CAPG = value.(CAPG).data,
        GEN = value.(GEN).data,
        CURTAIL = value.(CURTAIL).data,
        NSE = value.(NSE).data,
        GenInvCost = value(GenInvCost),
        TIC = value(TIC),
        EnergyProvisionCost = value(EnergyProvisionCost),
        UnservedEnergyCost = value(UnservedEnergyCost),
        CurtailedVreCost = value(CurtailedVreCost),
        OV = objective_value(model),
    )
end

case1scheme1 (generic function with 1 method)

In [3]:
c1s1b10 = case1scheme1(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=10e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
10077 rows, 7254 cols, 30282 nonzeros
10076 rows, 7141 cols, 30168 nonzeros
Presolve : Reductions: rows 10076(-4321); columns 7141(-1501); elements 30168(-5822)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     2.5399500000e+05 Pr: 1440(2.36784e+07) 0s
       2285     1.3001426396e+09 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 2285
Objective value     :  1.3001426396e+09
HiGHS run time      :          0.06


(CAPG = [47979.29032258065, 1840.1096774193518], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], CURTAIL = [-0.0 -0.0 … -0.0 -0.0; 0.0 0.0 … 0.0 0.0; 17712.800000000003 17067.800000000003 … 202.59999999999854 1117.5999999999985], NSE = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], GenInvCost = 1.0e10, TIC = 1.0e10, EnergyProvisionCost = 1.2803715362323117e9, UnservedEnergyCost = 0.0, CurtailedVreCost = 1.9771103400000032e7, OV = 1.300142639632312e9)

In [4]:
c1s1b20 = case1scheme1(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=20e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
10077 rows, 7254 cols, 30282 nonzeros
10076 rows, 7141 cols, 30168 nonzeros
Presolve : Reductions: rows 10076(-4321); columns 7141(-1501); elements 30168(-5822)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     2.5399500000e+05 Pr: 1440(2.36784e+07) 0s
       1441     9.3650015940e+08 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 1441
Objective value     :  9.3650015940e+08
HiGHS run time      :          0.02


(CAPG = [13144.666666666666, 49819.4], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], CURTAIL = [-0.0 -0.0 … -0.0 -0.0; 0.0 0.0 … 0.0 0.0; 17712.800000000003 17067.800000000003 … 202.59999999999854 1117.5999999999985], NSE = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], GenInvCost = 2.0e10, TIC = 2.0e10, EnergyProvisionCost = 9.16729056e8, UnservedEnergyCost = 0.0, CurtailedVreCost = 1.9771103400000032e7, OV = 9.365001594000006e8)

## Scheme 2 (Consider Storage)

Note: 
- 1 USD = 0.8 GBP
- 1 USD/kWh = 1000 USD/MWh
- Charging and Discharging cost: 5 USD/MWhh to promote efficient charging and discharging state

In [5]:
function case1scheme2(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
    budget, voll, bigm, epsilon)

    # SETS
    V = vreinfo.id
    G = generatorinfo.id
    S = storageinfo.id
    T = halfhourlydemand.hh
    T1 = T[2:end]

    # INITIATE MODEL
    model = Model()
    set_optimizer(model, HiGHS.Optimizer)

    # DECISION VARIABLES
    @variables(model, begin
        CAPG[G] >= 0
        CAPS[S] >= 0
        SOCM[S] >= 0
        GEN[G,T] >= 0
        SOC[S,T] >= 0
        CHARGE[S,T] >= 0
        DISCHARGE[S,T] >= 0
        CURTAIL[V,T] >= 0
        NSE[T] >= 0
    end)

    # BUDGET CONSTRAINT
    @expression(model, GenInvCost, sum(generatorinfo[generatorinfo.id.==g,:fixedCost][1] * CAPG[g] for g in G))
    @expression(model, StorPowInvCost, sum(0.8e3 * storageinfo[storageinfo.id.==s,:bestCasePowerCost][1] * CAPS[s] for s in S))
    @expression(model, StorEnInvCost, sum(0.8e3 * storageinfo[storageinfo.id.==s,:bestCaseEnergyCost][1] * SOCM[s] for s in S))
    @expression(model, TIC, GenInvCost + StorPowInvCost + StorEnInvCost)
    @constraint(model, cBudget, TIC <= budget)

    # OBJECTIVE FUNCTION
    @expression(model, EnergyProvisionCost, 
        sum(0.5 * generatorinfo[generatorinfo.id.==g,:varCost][1] * GEN[g,t] for g in G for t in T))
    @expression(model, UnservedEnergyCost, 
        sum(0.5 * voll * NSE[t] for t in T))
    @expression(model, CurtailedVreCost,
        sum(0.5 * vreinfo[vreinfo.id.==v,:varCost][1] * CURTAIL[v,t] for v in V for t in T))
    @expression(model, PenaltyChargeDischarge, 5 * sum(CHARGE[g,t] + DISCHARGE[g,t] for g in G for t in T))
    @objective(model, Min, EnergyProvisionCost + UnservedEnergyCost + CurtailedVreCost + PenaltyChargeDischarge)

    # POWER BALANCE CONSTRAINT
    @constraint(model, c01[t in T], sum(GEN[g,t] for g in G) + NSE[t] - sum(CURTAIL[v,t] for v in V) +
        sum(vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1] for v in V) +
        sum(DISCHARGE[s,t] for s in S) - sum(CHARGE[s,t] for s in S) == 
        halfhourlydemand[halfhourlydemand.hh.==t,:load][1])
    
    # GENERATOR LIMIT CONSTRAINT
    @constraint(model, c90[g in G, t in T], GEN[g,t] <= CAPG[g])

    # GENERATOR RAMP UP CONSTRAINT
    @constraint(model, c11[g in G, t in T1], 
        GEN[g,t] - GEN[g,t-1] <= generatorinfo[generatorinfo.id.==g,:rampupRate][1] * CAPG[g])
    
    # GENERATOR RAMP DOWN CONSTRAINT
    @constraint(model, c12[g in G, t in T1],
        GEN[g,t-1] - GEN[g,t] <= generatorinfo[generatorinfo.id.==g,:rampdownRate][1] * CAPG[g])
    
    # VRE CURTAIL CONSTRAINT
    @constraint(model, c13[v in V, t in T], CURTAIL[v,t] <= 
        vreinfo[vreinfo.id.==v,:installedMW][1] * 
        halfhourlyvrecf[(halfhourlyvrecf.hh.==t) .& (halfhourlyvrecf.vre_id.==v),:cf][1])
    
    # STORAGE CHARGE LIMIT CONSTRAINT
    @constraint(model, c92[s in S, t in T], CHARGE[s,t] <= CAPS[s])

    # STORAGE DISCHARGE LIMIT CONSTRAINT
    @constraint(model, c93[s in S, t in T], DISCHARGE[s,t] <= CAPS[s])
    
    # STORAGE SOC LIMIT CONSTRAINT
    @constraint(model, c19[s in S, t in T], SOC[s,t] <= SOCM[s])
    @constraint(model, c20[s in S], SOC[s,minimum(T)] == 0.5 * SOCM[s])
    @constraint(model, c21[s in S], SOC[s,maximum(T)] == 0.5 * SOCM[s])

    # STORAGE SOC UPDATE CONSTRAINT
    @constraint(model, c22[s in S, t in T1], SOC[s,t] == SOC[s,t-1] + 
        (CHARGE[s,t] * storageinfo[storageinfo.id.==s,:bestCaseEff][1]) - 
        (DISCHARGE[s,t] / storageinfo[storageinfo.id.==s,:bestCaseEff][1]))

    optimize!(model)

    return(
        CAPG = value.(CAPG).data,
        CAPS = value.(CAPS).data,
        SOCM = value.(SOCM).data,
        GEN = value.(GEN).data,
        SOC = value.(SOC).data,
        CHARGE = value.(CHARGE).data,
        DISCHARGE = value.(DISCHARGE).data,
        NET_DISCHARGE = value.(DISCHARGE).data .- value.(CHARGE).data,
        CURTAIL = value.(CURTAIL).data,
        NSE = value.(NSE).data,
        GenInvCost = value(GenInvCost),
        StorPowInvCost = value(StorPowInvCost),
        StorEnInvCost = value(StorEnInvCost),
        TIC = value(TIC),
        EnergyProvisionCost = value(EnergyProvisionCost),
        UnservedEnergyCost = value(UnservedEnergyCost),
        CurtailedVreCost = value(CurtailedVreCost),
        OV = objective_value(model),
    )
end

case1scheme2 (generic function with 1 method)

In [6]:
c1s2b10 = case1scheme2(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=10e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
90675 rows, 68156 cols, 272540 nonzeros
90675 rows, 68156 cols, 272540 nonzeros
Presolve : Reductions: rows 90675(-4376); columns 68156(-994); elements 272540(-5398)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 1440(3.86983e+08) 0s
       6360     4.3060335011e+04 Pr: 26280(3.21441e+11); Du: 0(6.76687e-07) 5s
      10814     1.5546269603e+05 Pr: 21061(1.13372e+11); Du: 0(6.2892e-07) 10s
      14348     2.3008790684e+05 Pr: 27870(1.86046e+12); Du: 0(4.61355e-07) 15s
      18190     3.8016557030e+05 Pr: 27687(8.08657e+10); Du: 0(6.1954e-07) 20s
      21899     4.8360309903e+05 Pr: 21920(9.94743e+10); Du: 0(6.12505e-07) 26s
      25063     6.1881039232e+05 Pr: 21056(2.21334e+11); Du: 0(6.12019e-07) 31s
      27993     7.6059610334e+05 Pr: 33986(1.63775e+13); Du: 0(5.70587e-07) 37s
      3060

(CAPG = [47979.290322580644, 1840.1096774193547], CAPS = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, 0.0, 0.0], SOCM = [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], SOC = [-0.0 0.0 … 0.0 -0.0; -0.0 -0.0 … -0.0 -0.0; … ; -0.0 -0.0 … -0.0 -0.0; -0.0 0.0 … -0.0 -0.0], CHARGE = [0.0 0.0 … 0.0 0.0; 0.0 -0.0 … -0.0 -0.0; … ; -0.0 0.0 … -0.0 -0.0; -0.0 -0.0 … -0.0 -0.0], DISCHARGE = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 -0.0 … 0.0 0.0; 0.0 -0.0 … 0.0 0.0], NET_DISCHARGE = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 -0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], CURTAIL = [-0.0 -0.0 … -0.0 -0.0; 0.0 0.0 … 0.0 0.0; 17712.800000000003 17067.800000000003 … 202.59999999999854 1117.5999999999985], NSE = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], GenInvCost = 1.0e10, StorPowInvCost = 0.0, StorEnInvCost = 0.0, TIC = 1.0e10, EnergyProvi

In [7]:
c1s2b20 = case1scheme2(vreinfo, generatorinfo, storageinfo, halfhourlydemand, halfhourlyvrecf; 
budget=20e9, voll=1e6, bigm=100e3, epsilon=0.05)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
90675 rows, 68156 cols, 272540 nonzeros
90675 rows, 68156 cols, 272540 nonzeros
Presolve : Reductions: rows 90675(-4376); columns 68156(-994); elements 272540(-5398)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 1440(3.86983e+08) 0s
       6688     4.1860773308e+04 Pr: 17178(1.12358e+11); Du: 0(6.08568e-07) 5s
      11204     1.4647886455e+05 Pr: 32413(9.82337e+11); Du: 0(4.09741e-07) 10s
      14816     2.1893981320e+05 Pr: 21580(8.87037e+11); Du: 0(6.25435e-07) 15s
      18108     2.7130323742e+05 Pr: 28409(1.43405e+11); Du: 0(9.40933e-07) 20s
      21354     3.0589790705e+05 Pr: 29336(3.49376e+11); Du: 0(7.89812e-07) 26s
      24466     3.4825597904e+05 Pr: 26595(3.05206e+11); Du: 0(1.11412e-06) 31s
      27376     3.8865359398e+05 Pr: 30926(7.1398e+11); Du: 0(1.07194e-06) 37s
      303

(CAPG = [4357.88057476665, 24147.563702701395], CAPS = [-0.0, 12915.264351236765, -0.0, -0.0, 2738.291371295227, 0.0, 0.0, 0.0, -0.0, -0.0, 5660.399999999995, -0.0, -0.0, -0.0], SOCM = [-0.0, 2.268530961275762e6, -0.0, -0.0, 199584.2600355829, -0.0, -0.0, -0.0, -0.0, -0.0, 66998.18181818399, 0.0, -0.0, -0.0], GEN = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], SOC = [-0.0 -0.0 … 0.0 -0.0; 1.134265480637881e6 1.1429187077532096e6 … 1.1335166886378808e6 1.134265480637881e6; … ; -0.0 -0.0 … -0.0 -0.0; -0.0 -0.0 … -0.0 -0.0], CHARGE = [0.0 0.0 … -0.0 -0.0; 0.0 12915.264351236765 … 202.59999999978035 1117.6000000001973; … ; -0.0 -0.0 … 0.0 0.0; -0.0 -0.0 … 0.0 0.0], DISCHARGE = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 -0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], NET_DISCHARGE = [0.0 0.0 … 0.0 0.0; 0.0 -12915.264351236765 … -202.59999999978035 -1117.6000000001973; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], CURTAIL = [-0.0 -0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; 9314.10862870478 1414.2442774680112 … 0.0 0.